In [1]:

import numpy as np
from sklearn.manifold import TSNE

# imports needed and set up logging
import gzip
import gensim 
import logging


In [3]:
#import gensim.downloader as api
#word2vec_model = api.load('word2vec-google-news-300')
import pandas as pd

In [5]:

!ls -lrt

total 37072
-rw-r--r--   1 klrao  staff      3047 Feb 19  2021 pipeLine-Ex.ipynb
-rw-r--r--@  1 klrao  staff    164733 Feb 19  2021 tweets.csv
-rw-r--r--   1 klrao  staff     12118 Sep 15 08:43 Practice-TweetAnalysis.ipynb
-rw-r--r--   1 klrao  staff     66065 Sep 15 10:53 Hands-On-NLP-Read-File.ipynb
-rw-r--r--   1 klrao  staff      8026 Sep 15 14:48 0.1.02-Sam-Ham-Basic-2Method.ipynb
-rw-r--r--   1 klrao  staff     51214 Sep 16 11:03 0.1.01-Sam-Ham-Basic-CollectionsCounter.ipynb
-rw-r--r--   1 klrao  staff     22076 Sep 16 20:08 MovieReviews.ipynb
-rw-r--r--   1 klrao  staff     71627 Sep 16 21:10 TextClassification.ipynb
-rw-r--r--   1 klrao  staff     17367 Sep 18 17:48 0.0.01-NLP-Basics.ipynb
drwxr-xr-x@  6 klrao  staff       192 Oct  2 11:15 enron1
drwx------  23 klrao  staff       736 Oct  3 17:10 archive
-rw-r--r--   1 klrao  staff     18403 Oct  3 17:49 0.0.03-LDA-Topic-Modelling.ipynb
-rw-r--r--   1 klrao  staff       836 Oct  3 22:11 0.0.999_filetext.ipynb
-rw-r--r--@  1 klr

In [8]:
import os
os.chdir('/Users/klrao/0.3.01_NLP/data/')

In [9]:
os.listdir()

['reviews_Cell_Phones_and_Accessories_5.json.gz',
 'Reviews.csv',
 'stackoverflow-data-idf.json',
 '.DS_Store',
 'concrete_data.csv',
 'news_category_dataset.json',
 'reviews_Cell_Phones_and_Accessories_5.json',
 'natural-language-data.txt',
 'moviereviews.tsv',
 'text1.txt',
 'stackoverflow-test.json',
 'archive.zip']

In [10]:
df = pd.read_json("reviews_Cell_Phones_and_Accessories_5.json",lines=True)
df.head()

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime
0,A30TL5EWN6DFXT,120401325X,christina,"[0, 0]",They look good and stick good! I just don't li...,4,Looks Good,1400630400,"05 21, 2014"
1,ASY55RVNIL0UD,120401325X,emily l.,"[0, 0]",These stickers work like the review says they ...,5,Really great product.,1389657600,"01 14, 2014"
2,A2TMXE2AFO7ONB,120401325X,Erica,"[0, 0]",These are awesome and make my phone look so st...,5,LOVE LOVE LOVE,1403740800,"06 26, 2014"
3,AWJ0WZQYMYFQ4,120401325X,JM,"[4, 4]",Item arrived in great time and was in perfect ...,4,Cute!,1382313600,"10 21, 2013"
4,ATX7CZYFXI1KW,120401325X,patrice m rogoza,"[2, 3]","awesome! stays on, and looks great. can be use...",5,leopard home button sticker for iphone 4s,1359849600,"02 3, 2013"


In [12]:
df.shape

(194439, 9)

In [13]:
df.reviewText

0         They look good and stick good! I just don't li...
1         These stickers work like the review says they ...
2         These are awesome and make my phone look so st...
3         Item arrived in great time and was in perfect ...
4         awesome! stays on, and looks great. can be use...
                                ...                        
194434    Works great just like my original one. I reall...
194435    Great product. Great packaging. High quality a...
194436    This is a great cable, just as good as the mor...
194437    I really like it becasue it works well with my...
194438    product as described, I have wasted a lot of m...
Name: reviewText, Length: 194439, dtype: object

In [14]:
df.reviewText[0]

"They look good and stick good! I just don't like the rounded shape because I was always bumping it and Siri kept popping up and it was irritating. I just won't buy a product like this again"

In [16]:
gensim.utils.simple_preprocess("They look good and stick good! I just don't like the rounded shape because I was always bumping it and Siri kept popping up and it was irritating. I just won't buy a product like this again")

['they',
 'look',
 'good',
 'and',
 'stick',
 'good',
 'just',
 'don',
 'like',
 'the',
 'rounded',
 'shape',
 'because',
 'was',
 'always',
 'bumping',
 'it',
 'and',
 'siri',
 'kept',
 'popping',
 'up',
 'and',
 'it',
 'was',
 'irritating',
 'just',
 'won',
 'buy',
 'product',
 'like',
 'this',
 'again']

In [17]:
review_text = df.reviewText.apply(gensim.utils.simple_preprocess)
review_text

0         [they, look, good, and, stick, good, just, don...
1         [these, stickers, work, like, the, review, say...
2         [these, are, awesome, and, make, my, phone, lo...
3         [item, arrived, in, great, time, and, was, in,...
4         [awesome, stays, on, and, looks, great, can, b...
                                ...                        
194434    [works, great, just, like, my, original, one, ...
194435    [great, product, great, packaging, high, quali...
194436    [this, is, great, cable, just, as, good, as, t...
194437    [really, like, it, becasue, it, works, well, w...
194438    [product, as, described, have, wasted, lot, of...
Name: reviewText, Length: 194439, dtype: object

In [19]:
model = gensim.models.Word2Vec(
    window=10,
    min_count=2,
    workers=4
)

In [20]:
model.build_vocab(review_text,progress_per=1000)

In [21]:
model.epochs

5

In [22]:
model.train(review_text,total_examples=model.corpus_count,epochs=model.epochs)

(61507064, 83868975)

In [23]:
model.save("./word2vec-amazon-cell-accessories-reviews-short.model")

In [25]:
model.wv.similarity(w1="great",w2="great")

1.0

In [26]:
model.wv.similarity(w1="cheap",w2="expensive")

0.47703198

In [28]:
model.wv['computer'].shape

(100,)